# Bug triage with Deep Learning - Doc2Vec

https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5

In [1]:
from __future__ import print_function, division

In [2]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [3]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

Using TensorFlow backend.


## Configurações Globais

In [4]:
MAX_SEQUENCE_LENGTH_T = 100 # 40
MAX_SEQUENCE_LENGTH_D = 500 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

### Parse bugs preproprecessed

In [5]:
# Domain to use
DOMAIN = 'openoffice'
METHOD = 'doc2vec'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs({})'.format(METHOD, DOMAIN)

# Test bugs
PATH_TEST = 'data/processed/{}/test.txt'.format(DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

## Auxiliary methods

In [6]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

#### Creating buckets and initialize the retrieval method

In [7]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [8]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


98070

#### Dicionário de títulos e descrições

In [9]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 36.3 s, sys: 1.36 s, total: 37.7 s
Wall time: 37 s


#### Get buckets for each issue

In [10]:
verbose = 0
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare dataset

In [11]:
experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')

Reading train data
Reading bug ids


#### Creating the queries to test

In [12]:
# Read and create the test queries duplicate
experiment.create_queries()

Reading queries from baseline.


#### Bugs train

In [13]:
bugs_train_ids = []
for bugs in experiment.baseline.train_data:
    bugs_train_ids.append(bugs[0])
    bugs_train_ids.append(bugs[1])

#### Corpus

In [14]:
lines_title = [baseline.bug_set[bug_id]['title'] for bug_id in bugs_train_ids]
lines_desc = [baseline.bug_set[bug_id]['description'] for bug_id in bugs_train_ids]

lines = []
for title, desc in zip(lines_title, lines_desc):
    lines.append(title)
    lines.append(desc)
    
tagged_data = [TaggedDocument(words=_d.split(' '), tags=[str(i)]) for i, _d in enumerate(lines)]

In [15]:
len(tagged_data)

214960

#### Train the method

In [16]:
%%time

import os

cpu = os.cpu_count() - 1

max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=epoch)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

#### Rank@25

In [17]:
len(experiment.retrieval.test)

3861

In [18]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

In [19]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, model, issues_by_buckets, 
                                                            bug_train_ids, method=METHOD)
"recall@25 last epoch:", recall

('recall@25 last epoch:', 0.23)

### Retrieval evaluation

In [20]:
print("Total of queries:", len(retrieval.test))

Total of queries: 3861


In [21]:
exported_rank[:20]

['32787:31730|25520:-0.012661933898925781,97062:-0.023519039154052734,21690:-0.025289177894592285,16386:-0.034046292304992676,16392:-0.03439128398895264,16384:-0.037656188011169434,89659:-0.04249119758605957,6682:-0.043840765953063965,70837:-0.04722702503204346,53634:-0.0511016845703125,7508:-0.05928456783294678,20778:-0.06839704513549805,50325:-0.0710761547088623,46770:-0.07538306713104248,25680:-0.07707440853118896,20431:-0.0785222053527832,45419:-0.07900810241699219,20776:-0.0802464485168457,20774:-0.0810089111328125,25224:-0.0831613540649414,20432:-0.08427608013153076,109993:-0.08461320400238037,50088:-0.08751988410949707,5906:-0.08933281898498535,62385:-0.09260118007659912,20406:-0.09293746948242188,84210:-0.09294211864471436,84328:-0.09346437454223633,71813:-0.09523487091064453',
 '31730:32787|33564:0.032472193241119385,21519:-0.059957265853881836,89652:-0.06834936141967773,18131:-0.08918654918670654,21887:-0.08922338485717773,24725:-0.09035122394561768,52452:-0.10249638557434082

#### Exporting the result

In [22]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))

In [23]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [24]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.19,
 '2 - recall_at_10': 0.2,
 '3 - recall_at_15': 0.21,
 '4 - recall_at_20': 0.22,
 '5 - recall_at_25': 0.23}

#### Saving model pretrained

In [25]:
model.save(os.path.join('modelos', SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(max_epochs))))
"Doc2vec model saved."

'Doc2vec model saved.'